In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
# split into training set and test
# in regression we skip this step in gridsearch method, but in classification we do this in order to get the confusion matrix and classification report.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],
              'max_features':['sqrt', 'log2', None],
              'n_estimators':[10, 100, 500]}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [10, 100, 500]},
             scoring='f1_weighted', verbose=3)

In [15]:
# print best parameter after tuning
# print(grid.best_params_)
clf=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [16]:
# f1 score is one of the evaluation metrics
# sample to check f1 macro value
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'log_loss', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9256751954513149


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 4 45]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [19]:
# roc_auc score is also an evaluation metric (a scoring parametr in classification)
# if the roc_auc value is nearly 1, then the model performs well

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9677070828331332

In [20]:
table=pd.DataFrame.from_dict(clf)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019265,0.001018,0.003415,0.000755,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.822092,0.850809,0.851527,0.870362,0.924528,0.863864,0.034029,25
1,0.163264,0.003461,0.007583,0.000666,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.888107,0.851527,0.925272,0.943041,0.891018,0.038400,12
2,0.810931,0.010183,0.029776,0.002646,gini,sqrt,500,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.907177,0.851527,0.925272,0.943699,0.899031,0.034648,3
3,0.018454,0.001823,0.003074,0.000929,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.864871,0.828959,0.832483,0.925272,0.883278,0.866972,0.035507,23
4,0.162928,0.001564,0.007610,0.000672,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.868752,0.851527,0.925272,0.962264,0.895059,0.041898,10
5,0.891891,0.023937,0.031789,0.001144,gini,log2,500,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.888107,0.851527,0.944161,0.962264,0.902708,0.043218,2
6,0.019921,0.001299,0.003212,0.001033,gini,None,10,"{'criterion': 'gini', 'max_features': None, 'n...",0.826263,0.849057,0.851527,0.888646,0.943699,0.871838,0.041122,18
7,0.195077,0.007007,0.007958,0.000491,gini,None,100,"{'criterion': 'gini', 'max_features': None, 'n...",0.847141,0.850809,0.851527,0.887907,0.905069,0.868491,0.023542,22
8,0.944675,0.007656,0.031624,0.001997,gini,None,500,"{'criterion': 'gini', 'max_features': None, 'n...",0.867478,0.850809,0.833323,0.887907,0.924528,0.872809,0.031542,16
9,0.019727,0.000788,0.003339,0.000864,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.868752,0.851527,0.886792,0.923510,0.875545,0.027774,15


In [21]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 20
Salary: 30000
Sex Male 0 or 1: 1


In [22]:
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
